## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import regex as re
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://www.amadorgov.org/services/covid-19/-fsiteid-1"

In [7]:
page = requests.get(url)

## Parse

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", id="widget_4_3778_3488")

Get timestamp

In [10]:
date_text = [x.parent for x in soup(text=re.compile(r"Current Status Updated "))][
    0
].text

In [11]:
date_text

'Current Status Updated 0ctober 1, 2021:'

In [12]:
date = date_text.replace("Current Status Updated ", "").replace(":", "")

In [13]:
latest_date = pd.to_datetime(date).date()

ParserError: Unknown string format: 0ctober 1, 2021

Get table

In [14]:
table = content.find("table")

In [15]:
tbody = soup.tbody

In [16]:
city_cell = tbody.select("tr")[1].select("td")[0]

In [17]:
cities = city_cell.find_all("p")

Parse the result

In [18]:
dict_list = []

In [19]:
for row in cities:
    city_text = unicodedata.normalize("NFKD", row.get_text())
    city_list = city_text.split(":")
    clean_list = [x.strip(" ") for x in city_list]
    d = dict(area=clean_list[0], confirmed_cases=clean_list[1])
    dict_list.append(d)

Convert to dataframe

In [20]:
df = pd.DataFrame(dict_list)
df.insert(0, "county", "Amador")
df["county_date"] = latest_date

In [21]:
df["area"] = df["area"].astype(str)

Mark the current date

In [22]:
tz = pytz.timezone("America/Los_Angeles")

In [23]:
today = datetime.now(tz).date()

In [24]:
slug = "amador"

## Vet

In [25]:
try:
    assert not len(df) > 7
except AssertionError:
    raise AssertionError("Amador's area scraper has extra rows")

In [26]:
try:
    assert not len(df) < 7
except AssertionError:
    raise AssertionError("Amador's area scraper is missing rows")

## Export

In [27]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [28]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [29]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [30]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [31]:
df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)